기본 공통 코드

In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


#Groq
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001E8191B6BD0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001E8196AE330> root_client=<openai.OpenAI object at 0x000001E81918C4A0> root_async_client=<openai.AsyncOpenAI object at 0x000001E819299220> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


문제 1-1 : 기본 체인 만들기 - AI 요리사
- 사용자가 재료를 입력하면 그 재료로 만들 수 있는 요리를 추천해주는 간단한 AI 요리사를 만들어보세요.


In [6]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    당신은 요리사입니다. 
    <Question>: {input} 의 재료로 만들 수 있는 요리 추천해주세요.
    출력으로 추천 요리와 간단한 레시피를 알려주세요.")
    """)                                     
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    당신은 요리사입니다. \n    <Question>: {input} 의 재료로 만들 수 있는 요리 추천해주세요.\n    출력으로 추천 요리와 간단한 레시피를 알려주세요.")\n    ')

In [8]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser
print(type(chain))

# chain 호출
try:
    result = chain.invoke({"input": "토마토, 양파, 치즈"})

    for token in result:
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.runnables.base.RunnableSequence'>
토마토, 양파, 치즈를 메인 재료로 사용하여 만들 수 있는 요리로 '토마토 치즈 구이'를 추천해 드립니다.

**재료:**
- 2-3개의 토마토
- 1/2개의 양파
- 1컵의 치즈 (모짜렐라 치즈 또는 체다 치즈 등)

**레시피:**
1. 토마토와 양파를 깨끗이 씻고, 토마토는 약 1cm 두께로 슬라이스하고, 양파는 얇게 슬라이스합니다.
2. 팬에 기름을 두르고 양파를 먼저 볶다가 투명한 색이 되면 토마토를 넣고 함께 볶습니다.
3. 소금과 후추로 간을 하고, 약간의 설탕을 추가해 토마토의 신맛을 중화시킵니다.
4. 볶은 토마토와 양파 혼합물을 오븐용 그릇에 옮기고, 위에 치즈를 뿌립니다.
5. 180도로 예열된 오븐에서 10-15분간 구워 치즈가 녹고 황금빛이 나도록 합니다.
6. 오븐에서 꺼내어 따뜻하게 서빙합니다.

**팁:**
- 치즈는 여러 종류를 혼합하여 사용할 수 있으며, 개인의 취향에 따라 조절할 수 있습니다.
- 신선한 바질이나 오레가노를 추가하면 더 풍부한 맛을 낼 수 있습니다.

이 요리는 간단하지만 맛이 뛰어나고, 영양가도 높아 가족이나 친구들과 함께하기 좋은 요리입니다.

문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템
- 사용자가 좋아하는 장르를 입력하면, 영화를 추천하고 그 영화의 정보(감독, 줄거리, 등장인물)들을  알려주는 2단계 체인을 구현해보세요.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트1
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# 프롬프트2
prompt2 = ChatPromptTemplate.from_template("""
            {movie} 추전한 영화의 제목을 먼저 알려주시고, 
            그 영화의 정보(감독, 등장인물)들을 알려주고, 줄거리를 3줄로 요약해줘
            정보의 줄거리는 "줄거리 : 1. 2. 3.내용" 으로 표현해주세요.
                                           """)

# Groq 사용
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 장르로 영화 추천
chain1 = prompt1 | llm | StrOutputParser()

# 체인 2: 영화 제목 → 요약 정보
chain2 = prompt2 | llm | StrOutputParser()

# 체인 2 : 감독, 등장인물, 줄거리 요약
try:
    chain2 = (
        {"movie": chain1}
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행
    response2 = chain2.invoke({"genre": "액션"}).strip()
    print(response2)
except Exception as e:
    print(f"오류 발생: {e}")

'다이 하드'입니다.

*   감독: 존 맥티어난
*   등장인물: 브루스 윌리스, 앨런 릭맨, 보니 베델리아

줄거리:

1.  뉴욕의 나카토미 빌딩에서 크리스마스 파티가 진행되는데, 테러리스트들이 빌딩을 점령하고 인질로 잡습니다.
2.  존 맥클레인은 아내를 찾으러 빌딩에 들어가게 되며, 테러리스트들과의 전투를 준비하게 됩니다.
3.  테러리스트들과의 치열한 전투와 액션 장면이 돋보이는 영화입니다.


문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용 
- 예시를 제공하는 PromptTemplate 사용하기

In [22]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
    "news": "네이버가 자사 검색 서비스를 대대적으로 개편하며, 생성형 인공지능 기술을 활용한 새로운 검색 알고리즘을 도입했다. 이는 구글의 검색 시장 독점에 도전하는 움직임으로, 국내외에서 AI 기반 검색 서비스 경쟁이 본격화될 전망이다.",
    "keywords": "네이버, 생성형 인공지능, 검색 알고리즘, 구글"
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{news}"),
        ("ai", "{keywords}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 뉴스 기사에서 핵심 키워드 3개를 추출하는 시스템입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."})
print(result.content)

구글, 제미나이, 인공지능
